In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from numpy import random
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import RDS password
from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-25 23:13:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.78MB/s    in 0.2s    

2021-05-25 23:13:55 (5.78 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [5]:
wineweather = pd.read_csv('/content/weather_scaled_updated.csv', index_col=0)
wineweather.head(4)

,winery,country,points,variety,winery_search,latitude,longitude,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,St. Julian,US,87,Riesling,St. Julian winery US,42.212251,-85.891713,283.486181,1016.253874,73.468022,0.118022
1,Sweet Cheeks,US,87,Pinot Noir,Sweet Cheeks winery US,43.956670,-123.279135,284.928462,1016.972170,75.501813,0.108269
2,Kirkland Signature,US,87,Cabernet Sauvignon,Kirkland Signature winery US,35.896713,-84.140546,288.217995,1017.277418,74.106346,0.129918
3,Louis M. Martini,US,87,Cabernet Sauvignon,Louis M. Martini winery US,38.491708,-122.452308,287.733599,1015.947830,74.851236,0.076868


In [6]:
wineweather.dtypes

winery                               object
country                              object
points                                int64
variety                              object
winery_search                        object
latitude                            float64
longitude                           float64
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
dtype: object

In [58]:
allwines = wineweather.drop(columns=['winery'])
uswinedata = wineweather[wineweather['country'] == 'US'].drop(columns=['winery','country'])
allwines['success'] = allwines.apply(lambda row: row.points >= 90, axis=1)
allwines.head(2)

,country,points,variety,winery_search,latitude,longitude,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm),success
0,US,87,Riesling,St. Julian winery US,42.212251,-85.891713,283.486181,1016.253874,73.468022,0.118022,False
1,US,87,Pinot Noir,Sweet Cheeks winery US,43.956670,-123.279135,284.928462,1016.972170,75.501813,0.108269,False


In [59]:
allwines.columns

Index(['country', 'points', 'variety', 'winery_search', 'latitude',
       'longitude', 'Average Temperature (Kelvin)',
       'Average Air Pressure (hPa)', 'Average Humidity (%)',
       'Average Daily Precipitation (mm)', 'success'],
      dtype='object')

In [62]:
county_weath = pd.read_csv('/content/uscounty_weather_v2.csv', index_col=0).dropna()
county_weath.head(2)

,zip,lat,lng,city,state_id,state_name,county_name,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,1001,42.06259,-72.62589,Agawam,MA,Massachusetts,Hampden,283.257967,1016.12956,70.273709,0.126154
1,1002,42.37492,-72.46210,Amherst,MA,Massachusetts,Hampshire,283.257967,1016.12956,70.273709,0.126154


In [63]:
county_weath.dtypes

zip                                   int64
lat                                 float64
lng                                 float64
city                                 object
state_id                             object
state_name                           object
county_name                          object
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
dtype: object

In [64]:
len(county_weath)

1796

In [65]:
county_weath.columns

Index(['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'county_name',
       'Average Temperature (Kelvin)', 'Average Air Pressure (hPa)',
       'Average Humidity (%)', 'Average Daily Precipitation (mm)'],
      dtype='object')

In [66]:
county_ids = county_weath.drop(columns=['lat','lng','Average Temperature (Kelvin)', 'Average Air Pressure (hPa)',
       'Average Humidity (%)', 'Average Daily Precipitation (mm)'])
county_ids.head(2)

,zip,city,state_id,state_name,county_name
0,1001,Agawam,MA,Massachusetts,Hampden
1,1002,Amherst,MA,Massachusetts,Hampshire


In [67]:
bigcounty = county_weath.drop(columns=['city', 'state_id', 'state_name', 'county_name'])
bigcounty.head(2)

,zip,lat,lng,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,1001,42.06259,-72.62589,283.257967,1016.12956,70.273709,0.126154
1,1002,42.37492,-72.46210,283.257967,1016.12956,70.273709,0.126154


In [68]:
bigcounty.dtypes

zip                                   int64
lat                                 float64
lng                                 float64
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
dtype: object

In [69]:
countydrops = ['zip']

In [70]:
mer_df = allwines[allwines['variety'] == 'Merlot']
syr_df = allwines[allwines['variety'] == 'Syrah']
savyb_df = allwines[allwines['variety'] == 'Sauvignon Blanc']
ries_df = allwines[allwines['variety'] == 'Riesling']
cabsav_df = allwines[allwines['variety'] == 'Cabernet Sauvignon']
char_df = allwines[allwines['variety'] == 'Chardonnay']
pino_df = allwines[allwines['variety'] == 'Pinot Noir']

In [71]:
winedfs = [mer_df,syr_df,savyb_df,ries_df,cabsav_df,char_df,pino_df]
for x in winedfs:
  print(len(x))

2983
3944
4729
5110
8837
11334
12681


In [72]:
mer_df.dtypes

country                              object
points                                int64
variety                              object
winery_search                        object
latitude                            float64
longitude                           float64
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
success                                bool
dtype: object

In [73]:
mer_df.columns

Index(['country', 'points', 'variety', 'winery_search', 'latitude',
       'longitude', 'Average Temperature (Kelvin)',
       'Average Air Pressure (hPa)', 'Average Humidity (%)',
       'Average Daily Precipitation (mm)', 'success'],
      dtype='object')

In [75]:
winedropcols = ['country', 'points', 'variety', 'winery_search','success']

In [76]:
#features and targets
variety = mer_df.copy()
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [77]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.900804289544236

In [78]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
mer_preds = rf.predict(Xcount)

In [79]:
#features and targets
variety = syr_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [80]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.821501014198783

In [81]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
syr_preds = rf.predict(Xcount)

In [82]:
#features and targets
variety = savyb_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [83]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.8605240912933221

In [84]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
savyb_preds = rf.predict(Xcount)

In [85]:
#features and targets
variety = ries_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [86]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.797339593114241

In [87]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
ries_preds = rf.predict(Xcount)

In [88]:
#features and targets
variety = cabsav_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [89]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.8235294117647058

In [90]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
cabsav_preds = rf.predict(Xcount)

In [91]:
#features and targets
variety = char_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [92]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.8274523641496119

In [93]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
char_preds = rf.predict(Xcount)

In [94]:
#features and targets
variety = pino_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [95]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7984862819299905

In [96]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
pino_preds = rf.predict(Xcount)

In [115]:
len(bigcounty)

1796

In [116]:
counties_preds = bigcounty.copy()
counties_preds['mer_success'] = mer_preds
counties_preds['syr_success'] = syr_preds
counties_preds['savyb_success'] = savyb_preds
counties_preds['ries_success'] = ries_preds
counties_preds['cabsav_success'] = cabsav_preds
counties_preds['char_success'] = char_preds
counties_preds['pinot_success'] = pino_preds
counties_link = counties_preds.merge(county_ids, how='inner', on='zip')
len(counties_link)

1796

In [117]:
counties_link.columns

Index(['zip', 'lat', 'lng', 'Average Temperature (Kelvin)',
       'Average Air Pressure (hPa)', 'Average Humidity (%)',
       'Average Daily Precipitation (mm)', 'mer_success', 'syr_success',
       'savyb_success', 'ries_success', 'cabsav_success', 'char_success',
       'pinot_success', 'city', 'state_id', 'state_name', 'county_name'],
      dtype='object')

In [139]:
counties_link.to_csv('county_linearpreds_mktwo.csv')
counties_link.to_json('county_linearpreds_mktwo.json')

In [118]:
machinecols = [ 'mer_success', 'syr_success',
       'savyb_success', 'ries_success', 'cabsav_success', 'char_success',
       'pinot_success']
for x in machinecols:
  print(counties_link[x].value_counts())

False    1663
True      133
Name: mer_success, dtype: int64
False    1461
True      335
Name: syr_success, dtype: int64
False    1610
True      186
Name: savyb_success, dtype: int64
False    1741
True       55
Name: ries_success, dtype: int64
False    1554
True      242
Name: cabsav_success, dtype: int64
False    1619
True      177
Name: char_success, dtype: int64
False    1474
True      322
Name: pinot_success, dtype: int64


In [122]:
calcpreds = counties_link.copy()
calcpreds['goodwines'] = calcpreds.apply(lambda x: sum(x['mer_success':'pinot_success']), axis=1)
calcpreds['goodwines'].value_counts()

1    732
0    729
2    289
3     44
4      2
Name: goodwines, dtype: int64

In [123]:
def statecalc(bigdf, places):
  statewins = []
  for place in places:
    winvars = 0
    mer = 0
    syr = 0
    savyb = 0
    ries = 0
    cabsav = 0
    char = 0
    pinot = 0
    state_df = bigdf[bigdf['state_name'] == place]
    winvars = sum(state_df['goodwines'])
    mer = sum(state_df['mer_success'])
    syr = sum(state_df['syr_success'])
    savyb = sum(state_df['savyb_success'])
    ries = sum(state_df['ries_success'])
    cabsav = sum(state_df['cabsav_success'])
    char = sum(state_df['char_success'])
    pinot = sum(state_df['pinot_success'])
    statewins.append([place, winvars, mer, syr, savyb, ries, cabsav, char, pinot])
  return pd.DataFrame(statewins, columns=['state','tot_success','merlot','syrah','savyb','riesling','cabsav','chard','pinot'])

In [133]:
sumchart = statecalc(calcpreds, calcpreds['state_name'].unique())


In [134]:
sumchart.columns

Index(['state', 'tot_success', 'merlot', 'syrah', 'savyb', 'riesling',
       'cabsav', 'chard', 'pinot'],
      dtype='object')

In [135]:
sumchart.sort_values('tot_success').tail(10)

,state,tot_success,merlot,syrah,savyb,riesling,cabsav,chard,pinot
21,Ohio,45,0,3,10,2,6,10,14
28,North Dakota,47,32,0,15,0,0,0,0
26,Minnesota,48,28,0,10,4,0,6,0
20,Kentucky,50,0,0,0,3,26,17,4
23,Michigan,53,2,6,0,3,11,15,16
31,Missouri,54,0,10,1,0,4,16,23
12,West Virginia,54,0,0,22,0,5,11,16
10,Virginia,68,0,38,1,0,9,0,20
37,Texas,131,0,11,18,0,34,10,58
15,Georgia,152,5,79,0,0,35,7,26


In [136]:
sumchart.to_json('state_sums_classifier_mktwo.json')

In [100]:
#counties_preds.to_csv('county_predictions.csv')

In [101]:
#spark_counties = spark.createDataFrame(counties_preds)
#spark_counties.show()

In [102]:
#spark_counties.write.jdbc(url=jdbc_url, table='county_predictions', mode=mode, properties=config)

In [103]:
#sparked_preds = spark.read.jdbc(url=jdbc_url, table='county_predictions', properties=config)

In [104]:
#sparked_preds.show()

In [105]:
#winners_df = pd.DataFrame(sparked_preds.select('zip','lat','lng','state_id','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success').collect(), columns=['zip','lat','lng','state','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success'], dtype=float)
#winners_df.describe()

In [106]:
#winners_df.columns

In [107]:
#winners_df

In [108]:
#winners_df['mer_success'].value_counts()